# ROCKET

# Overview

ROCKET [1] transforms time series using random convolutional kernels (random length, weights, bias, dilation, and padding). ROCKET computes two features from the resulting feature maps: the max, and the proportion of positive values (or ppv). The transformed features are used to train a linear classifier.

[1] Dempster A, Petitjean F, Webb GI (2019) ROCKET: Exceptionally fast and accurate time series classification using random convolutional kernels. [arXiv:1910.13051](https://arxiv.org/abs/1910.13051)

***

## Imports

**Note**: ROCKET compiles (via Numba) on import, which may take a few seconds.

In [1]:
import numpy as np

from sktime.datasets import load_gunpoint
from sktime.datasets.base import load_japanese_vowels

from sktime.transformers.rocket import Rocket
from sklearn.linear_model import RidgeClassifierCV

***

# 1 Univariate Time Series

## 1.1 Load the Training Data

In [2]:
X_training, Y_training = load_gunpoint(split = "TRAIN", return_X_y = True)

## 1.2 Initialise ROCKET and Transform the Training Data

In [3]:
rocket = Rocket() # by default, ROCKET uses 10,000 kernels

rocket.fit(X_training)

X_training_transform = rocket.transform(X_training)

## 1.3 Fit a Classifier

We recommend using `RidgeClassifierCV` from scikit-learn for smaller datasets (fewer than approx. 20K training examples), and using logistic regression trained using stochastic gradient descent for larger datasets.

In [4]:
classifier = RidgeClassifierCV(alphas = np.logspace(-3, 3, 10),
                               normalize = True)

classifier.fit(X_training_transform, Y_training)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                  class_weight=None, cv=None, fit_intercept=True,
                  normalize=True, scoring=None, store_cv_values=False)

## 1.4 Load and Transform the Test Data

In [5]:
X_test, Y_test = load_gunpoint(split = "TEST", return_X_y = True)

X_test_transform = rocket.transform(X_test)

## 1.5 Classify the Test Data

In [6]:
classifier.score(X_test_transform, Y_test)

1.0

***

# 2 Multivariate Time Series

## 2.1 Load the Training Data

In [7]:
X_training, Y_training = load_japanese_vowels(split = "TRAIN", return_X_y = True)

## 2.2 Initialise ROCKET and Transform the Training Data

In [8]:
rocket = Rocket() # by default, ROCKET uses 10,000 kernels

rocket.fit(X_training)

X_training_transform = rocket.transform(X_training)

## 2.3 Fit a Classifier

We recommend using `RidgeClassifierCV` from scikit-learn for smaller datasets (approx. fewer than 20K training examples), and using logistic regression trained using stochastic gradient descent for larger datasets.

In [9]:
classifier = RidgeClassifierCV(alphas = np.logspace(-3, 3, 10),
                               normalize = True)

classifier.fit(X_training_transform, Y_training)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                  class_weight=None, cv=None, fit_intercept=True,
                  normalize=True, scoring=None, store_cv_values=False)

## 2.4 Load and Transform the Test Data

In [10]:
X_test, Y_test = load_japanese_vowels(split = "TEST", return_X_y = True)

X_test_transform = rocket.transform(X_test)

## 2.5 Classify the Test Data

In [11]:
classifier.score(X_test_transform, Y_test)

1.0